In [ ]:
import numpy as np
from datasets import Dataset

In [1]:
videos = Dataset.load_from_disk('../data/videos')
labels = np.load('../data/cluster_labels.npy')

NameError: name 'Dataset' is not defined